In [244]:
import pandas as pd
import numpy as np
import math

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly.offline as pyo




# Data Retrieval & Cleaning

In [245]:
quarterly_data_path = 'data/quarterly_data.csv'
quarterly_data = pd.read_csv(quarterly_data_path)
quarterly_data.dropna(inplace=True)


In [246]:
projects_by_vol_path = 'data/projects_by_vol.csv'
projects_by_vol = pd.read_csv(projects_by_vol_path)
projects_by_vol.dropna(inplace=True)


In [247]:
quarterly_data['QUARTER'] = quarterly_data['QUARTER'].apply(lambda x: x.strip())

# Convert 'QUARTER' column to datetime, with error handling
quarterly_data['QUARTER'] = pd.to_datetime(quarterly_data['QUARTER'], errors='coerce')

In [248]:
quarterly_data.set_index('QUARTER', inplace=True)


In [249]:
quarterly_totals = quarterly_data.groupby('QUARTER')['VOLUME_USD'].sum()
quarterly_data['PERCENTAGE_OF_TOTAL_VOLUME'] = quarterly_data.apply(
    lambda row: (row['VOLUME_USD'] / quarterly_totals[row.name]) * 100,
    axis=1
)

quarterly_totals = quarterly_data.groupby('QUARTER')['TOTAL_FEES_USD'].sum()
quarterly_data['PERCENTAGE_OF_TOTAL_FEES'] = quarterly_data.apply(
    lambda row: (row['TOTAL_FEES_USD'] / quarterly_totals[row.name]) * 100,
    axis=1
)

# Print the DataFrame with the new column for verification
print(quarterly_data[['NETWORK', 'PERCENTAGE_OF_TOTAL_VOLUME','PERCENTAGE_OF_TOTAL_FEES']])

             NETWORK  PERCENTAGE_OF_TOTAL_VOLUME  PERCENTAGE_OF_TOTAL_FEES
QUARTER                                                                   
2024-04-01  Optimism                    0.444831                  0.086662
2024-04-01      Base                   36.747597                 29.582075
2024-04-01  Arbitrum                    1.860435                  2.581483
2024-04-01  Ethereum                   60.947137                 67.749780
2024-01-01  Arbitrum                    2.128596                  2.801301
2024-01-01  Ethereum                   91.037882                 89.451095
2024-01-01  Optimism                    0.192733                  0.233034
2024-01-01      Base                    6.640788                  7.514569
2023-10-01      Base                    0.281792                  0.484509
2023-10-01  Arbitrum                    1.607893                  2.817077
2023-10-01  Optimism                    0.068783                  0.138305
2023-10-01  Ethereum     

In [250]:
quarterly_totals_layers = quarterly_data.groupby(['QUARTER', 'LAYER'])[['VOLUME_USD', 'TOTAL_FEES_USD', 'UNIQUE_TOTAL_USERS']].sum()
quarterly_totals_layers = quarterly_totals_layers.reset_index()

quarterly_totals_2 = quarterly_totals_layers.groupby('QUARTER')['TOTAL_FEES_USD'].sum()

# Calculate percentage of total fees
quarterly_totals_layers['PERCENTAGE_OF_TOTAL_FEES'] = quarterly_totals_layers.apply(
    lambda row: (row['TOTAL_FEES_USD'] / quarterly_totals_2[row['QUARTER']]) * 100,
    axis=1
)

print(quarterly_totals_layers)

     QUARTER LAYER    VOLUME_USD  TOTAL_FEES_USD  UNIQUE_TOTAL_USERS  \
0 2023-07-01    L1  2.108915e+08    9.204380e+06            245918.0   
1 2023-07-01    L2  2.993722e+06    2.444479e+05            111767.0   
2 2023-10-01    L1  3.372263e+08    1.018415e+07            219847.0   
3 2023-10-01    L2  6.736397e+06    3.628039e+05             91174.0   
4 2024-01-01    L1  3.134141e+08    1.056557e+07            207228.0   
5 2024-01-01    L2  3.085369e+07    1.245990e+06            116469.0   
6 2024-04-01    L1  1.464951e+08    4.000557e+06            119206.0   
7 2024-04-01    L2  9.386908e+07    1.904343e+06            254842.0   

   PERCENTAGE_OF_TOTAL_FEES  
0                 97.412929  
1                  2.587071  
2                 96.560109  
3                  3.439891  
4                 89.451095  
5                 10.548905  
6                 67.749780  
7                 32.250220  


In [251]:
quarterly_data = quarterly_data.reset_index()

In [252]:
quarterly_data['QUARTER_PERIOD'] = quarterly_data['QUARTER'].dt.to_period('Q')
quarterly_data['QUARTER_STR'] = quarterly_data['QUARTER_PERIOD'].apply(lambda p: f"{p.year} Q{p.quarter}")


In [253]:
quarterly_totals_layers['CUMULATIVE_FEES'] = quarterly_totals_layers.groupby(['LAYER', 'QUARTER'])['TOTAL_FEES_USD'].cumsum()
quarterly_totals_layers

,QUARTER,LAYER,VOLUME_USD,TOTAL_FEES_USD,UNIQUE_TOTAL_USERS,PERCENTAGE_OF_TOTAL_FEES,CUMULATIVE_FEES
0,2023-07-01,L1,2.108915e+08,9.204380e+06,245918.0,97.412929,9.204380e+06
1,2023-07-01,L2,2.993722e+06,2.444479e+05,111767.0,2.587071,2.444479e+05
2,2023-10-01,L1,3.372263e+08,1.018415e+07,219847.0,96.560109,1.018415e+07
3,2023-10-01,L2,6.736397e+06,3.628039e+05,91174.0,3.439891,3.628039e+05
4,2024-01-01,L1,3.134141e+08,1.056557e+07,207228.0,89.451095,1.056557e+07
5,2024-01-01,L2,3.085369e+07,1.245990e+06,116469.0,10.548905,1.245990e+06
6,2024-04-01,L1,1.464951e+08,4.000557e+06,119206.0,67.749780,4.000557e+06
7,2024-04-01,L2,9.386908e+07,1.904343e+06,254842.0,32.250220,1.904343e+06


# Visualizations

In [254]:
color_map = {
    'Base': 'blue',
    'Arbitrum': 'red',
    'Ethereum': '#6cfdd2',
    'Optimism': 'purple',
    'L1': 'orange',
    'L2': 'green'
}

legend_order = ['Arbitrum', 'Base','Ethereum', 'Optimism']


In [255]:
quarterly_data

,QUARTER,VOLUME_USD,TOTAL_FEES_USD,UNIQUE_TOTAL_USERS,NETWORK,LAYER,PERCENTAGE_OF_TOTAL_VOLUME,PERCENTAGE_OF_TOTAL_FEES,QUARTER_PERIOD,QUARTER_STR
0,2024-04-01,1.069214e+06,5.117316e+03,8621.0,Optimism,L2,0.444831,0.086662,2024Q2,2024 Q2
1,2024-04-01,8.832805e+07,1.746792e+06,224213.0,Base,L2,36.747597,29.582075,2024Q2,2024 Q2
2,2024-04-01,4.471819e+06,1.524340e+05,22008.0,Arbitrum,L2,1.860435,2.581483,2024Q2,2024 Q2
3,2024-04-01,1.464951e+08,4.000557e+06,119206.0,Ethereum,L1,60.947137,67.749780,2024Q2,2024 Q2
4,2024-01-01,7.328073e+06,3.308773e+05,27181.0,Arbitrum,L2,2.128596,2.801301,2024Q1,2024 Q1
5,2024-01-01,3.134141e+08,1.056557e+07,207228.0,Ethereum,L1,91.037882,89.451095,2024Q1,2024 Q1
6,2024-01-01,6.635193e+05,2.752500e+04,15267.0,Optimism,L2,0.192733,0.233034,2024Q1,2024 Q1
7,2024-01-01,2.286210e+07,8.875876e+05,74021.0,Base,L2,6.640788,7.514569,2024Q1,2024 Q1
8,2023-10-01,9.692585e+05,5.110091e+04,23088.0,Base,L2,0.281792,0.484509,2023Q4,2023 Q4
9,2023-10-01,5.530552e+06,2.971159e+05,53935.0,Arbitrum,L2,1.607893,2.817077,2023Q4,2023 Q4


In [256]:
quarterly_data

,QUARTER,VOLUME_USD,TOTAL_FEES_USD,UNIQUE_TOTAL_USERS,NETWORK,LAYER,PERCENTAGE_OF_TOTAL_VOLUME,PERCENTAGE_OF_TOTAL_FEES,QUARTER_PERIOD,QUARTER_STR
0,2024-04-01,1.069214e+06,5.117316e+03,8621.0,Optimism,L2,0.444831,0.086662,2024Q2,2024 Q2
1,2024-04-01,8.832805e+07,1.746792e+06,224213.0,Base,L2,36.747597,29.582075,2024Q2,2024 Q2
2,2024-04-01,4.471819e+06,1.524340e+05,22008.0,Arbitrum,L2,1.860435,2.581483,2024Q2,2024 Q2
3,2024-04-01,1.464951e+08,4.000557e+06,119206.0,Ethereum,L1,60.947137,67.749780,2024Q2,2024 Q2
4,2024-01-01,7.328073e+06,3.308773e+05,27181.0,Arbitrum,L2,2.128596,2.801301,2024Q1,2024 Q1
5,2024-01-01,3.134141e+08,1.056557e+07,207228.0,Ethereum,L1,91.037882,89.451095,2024Q1,2024 Q1
6,2024-01-01,6.635193e+05,2.752500e+04,15267.0,Optimism,L2,0.192733,0.233034,2024Q1,2024 Q1
7,2024-01-01,2.286210e+07,8.875876e+05,74021.0,Base,L2,6.640788,7.514569,2024Q1,2024 Q1
8,2023-10-01,9.692585e+05,5.110091e+04,23088.0,Base,L2,0.281792,0.484509,2023Q4,2023 Q4
9,2023-10-01,5.530552e+06,2.971159e+05,53935.0,Arbitrum,L2,1.607893,2.817077,2023Q4,2023 Q4


In [257]:
quarterly_data_sorted_vol = quarterly_data.sort_values(by='VOLUME_USD', ascending=False)
# quarterly_data_sorted_vol = quarterly_data_sorted_vol.sort_index(ascending=True)
quarterly_layer_vol = quarterly_data.groupby(['QUARTER','LAYER']).sum(numeric_only=True)
quarterly_layer_vol.reset_index(inplace=True)
quarterly_layer_vol['QUARTER_PERIOD'] = quarterly_layer_vol['QUARTER'].dt.to_period('Q')
quarterly_layer_vol['QUARTER_STR'] = quarterly_layer_vol['QUARTER_PERIOD'].apply(lambda p: f"{p.year} Q{p.quarter}")
quarterly_layer_vol.set_index('QUARTER', inplace=True)

In [258]:
def clean_values(x):
    if x >= 1e9:
        return f'{x/1e9:.1f}B'  # Billion
    elif x >= 1e6:
        return f'{x/1e6:.1f}M'  # Million
    elif x >= 1e3:
        return f'{x/1e3:.1f}K'  # Thousand
    else:
        return str(x)

In [259]:
quarterly_data['VOLUME_USD_CLEAN'] = quarterly_data['VOLUME_USD'].apply(clean_values)

In [260]:
quarterly_data_sorted_vol = quarterly_data_sorted_vol.sort_values('QUARTER', ascending=False)

In [261]:
quarterly_data_sorted_vol.columns

Index(['QUARTER', 'VOLUME_USD', 'TOTAL_FEES_USD', 'UNIQUE_TOTAL_USERS',
       'NETWORK', 'LAYER', 'PERCENTAGE_OF_TOTAL_VOLUME',
       'PERCENTAGE_OF_TOTAL_FEES', 'QUARTER_PERIOD', 'QUARTER_STR'],
      dtype='object')

In [262]:
fig = go.Figure()

# Plotting the bar chart for volume by network
for network in legend_order:
    network_df = quarterly_data_sorted_vol[quarterly_data_sorted_vol['NETWORK'] == network]
    fig.add_trace(
        go.Bar(
            y=network_df['QUARTER_STR'],
            x=network_df['VOLUME_USD'],
            name=network,
            orientation='h',
            text=network_df['VOLUME_USD'].apply(clean_values),
            textposition='auto',
            marker=dict(color=color_map.get(network, 'black')),
            textfont=dict(size=25, family="Arial, sans-serif")
        )
    )

# Layout
fig.update_layout(
    height=800,
    barmode='group',
    yaxis=dict(title='Sales Volume (USD)'),
    xaxis=dict(title='Quarter'),
    legend=dict(yanchor="top", y=0.5, xanchor="left", x=1.05),
    plot_bgcolor='#f4f4f4',
    paper_bgcolor='#f4f4f4',
    margin=dict(t=100, b=50, l=50, r=50),
    uniformtext=dict(mode="show", minsize=21),  # Adjust minsize as needed
    font=dict(size=25, family="Arial, sans-serif"),  # General font size, can be adjusted as needed
    # title={
    #     'text': 'OpenSea Volume Analysis by Network',
    #     'font': {
    #         'family': 'Arial, sans-serif',  # Font family
    #         'size': 24,  # Font size
    #         'color': 'Black'  # Font color
    #     }
    # }
)

# Adjust layout size
fig.update_layout(
    width=1500,  # Adjust as needed
    height=800,  # Adjust as needed
    margin=dict(
        l=50,  # Left margin
        r=50,  # Right margin
        t=100,  # Top margin
        b=50   # Bottom margin
    )
)

fig.update_layout(
    xaxis_title=dict(
        text="Sales Volume (USD)",
        font=dict(size=25, family="Arial, sans-serif", weight="bold", color='black')
    ),
    yaxis_title=dict(
        text="Quarter",
        font=dict(size=25, family="Arial, sans-serif", weight="bold", color='black')
    )
)

# Figure
pyo.iplot(fig)
pio.write_image(fig, 'img/OpenSea Volume Analysis by Network.png', scale=3)


In [263]:
fig = make_subplots(
    rows=2, cols=1, 
    shared_xaxes=False,
    subplot_titles=("OpenSea Volume by Layer", "OpenSea Volume by Network"),
    row_heights=[0.6, 0.4]
)

for network in legend_order:
    network_df = quarterly_data_sorted_vol[quarterly_data_sorted_vol['NETWORK'] == network]
    fig.add_trace(
        go.Bar(
            y=network_df['QUARTER_STR'],
            x=network_df['VOLUME_USD'],
            name=network,
            orientation='h',
            text=network_df['VOLUME_USD'].apply(clean_values),
            textposition='outside',
            marker=dict(color=color_map.get(network, 'black'))
        ),
        row=2, col=1   
    )

for layer in quarterly_layer_vol['LAYER'].unique():
    layer_df = quarterly_layer_vol[quarterly_layer_vol['LAYER'] == layer]
    fig.add_trace(
        go.Bar(
            x=layer_df['QUARTER_STR'],  # QUARTER
            y=layer_df['VOLUME_USD'],
            name=f'{layer}',
            text=layer_df['VOLUME_USD'].apply(clean_values),
            textposition='outside',
            marker=dict(color=color_map.get(layer, 'black'))
            # showlegend=not show_legend.get(layer, False)
        ),
        row=1, col=1
    )
    # show_legend[layer] = True
    
    

# Layout
fig.update_layout(
    height=800,
    barmode='group',
    yaxis=dict(title='Sales Volume (USD)'),
    xaxis=dict(title='Quarter'),
    legend=dict(yanchor="top", y=0.5, xanchor="left", x=1.05),
    plot_bgcolor='#f4f4f4',
    paper_bgcolor='#f4f4f4',
    margin=dict(t=100, b=50, l=50, r=50)    
)

fig.update_xaxes(title='Sales Volume (USD)', row=2, col=1)
fig.update_yaxes(title='Quarter', row=2, col=1)
fig.update_layout(
    uniformtext=dict(mode="show", minsize=20),  # Adjust minsize as needed
    font=dict(size=20)  # General font size, can be adjusted as needed
)

# fig.update_layout(
#     title={
#         'text': 'OpenSea Volume Analysis by Network and Layer',
#         'font': {
#             'family': 'Arial, sans-serif',  # You can specify the font family
#             'size': 24,  # Adjust the font size as needed
#             'color': 'Black'  # Adjust the font color as needed
#         }
#     }
# )


for i, yaxis in enumerate(fig.select_yaxes(), 1):
    legend_name = f"legend{i}"
    fig.update_layout({legend_name: dict(y=yaxis.domain[1], yanchor="bottom")}, showlegend=True)
    fig.update_traces(row=i, legend=legend_name)

fig.update_layout(
    width=1200,  # Adjust as needed
    height=800,  # Adjust as needed
    margin=dict(
        l=50,  # Left margin
        r=50,  # Right margin
        t=100,  # Top margin
        b=50   # Bottom margin
    )
)
# Figure

pyo.iplot(fig)
# pio.write_image(fig, 'OpenSea Volume Analysis by Network and Layer.png', scale=3)


In [264]:
quarterly_totals_layers

,QUARTER,LAYER,VOLUME_USD,TOTAL_FEES_USD,UNIQUE_TOTAL_USERS,PERCENTAGE_OF_TOTAL_FEES,CUMULATIVE_FEES
0,2023-07-01,L1,2.108915e+08,9.204380e+06,245918.0,97.412929,9.204380e+06
1,2023-07-01,L2,2.993722e+06,2.444479e+05,111767.0,2.587071,2.444479e+05
2,2023-10-01,L1,3.372263e+08,1.018415e+07,219847.0,96.560109,1.018415e+07
3,2023-10-01,L2,6.736397e+06,3.628039e+05,91174.0,3.439891,3.628039e+05
4,2024-01-01,L1,3.134141e+08,1.056557e+07,207228.0,89.451095,1.056557e+07
5,2024-01-01,L2,3.085369e+07,1.245990e+06,116469.0,10.548905,1.245990e+06
6,2024-04-01,L1,1.464951e+08,4.000557e+06,119206.0,67.749780,4.000557e+06
7,2024-04-01,L2,9.386908e+07,1.904343e+06,254842.0,32.250220,1.904343e+06


In [265]:
quarterly_data_sorted_vol

,QUARTER,VOLUME_USD,TOTAL_FEES_USD,UNIQUE_TOTAL_USERS,NETWORK,LAYER,PERCENTAGE_OF_TOTAL_VOLUME,PERCENTAGE_OF_TOTAL_FEES,QUARTER_PERIOD,QUARTER_STR
3,2024-04-01,1.464951e+08,4.000557e+06,119206.0,Ethereum,L1,60.947137,67.749780,2024Q2,2024 Q2
1,2024-04-01,8.832805e+07,1.746792e+06,224213.0,Base,L2,36.747597,29.582075,2024Q2,2024 Q2
2,2024-04-01,4.471819e+06,1.524340e+05,22008.0,Arbitrum,L2,1.860435,2.581483,2024Q2,2024 Q2
0,2024-04-01,1.069214e+06,5.117316e+03,8621.0,Optimism,L2,0.444831,0.086662,2024Q2,2024 Q2
5,2024-01-01,3.134141e+08,1.056557e+07,207228.0,Ethereum,L1,91.037882,89.451095,2024Q1,2024 Q1
7,2024-01-01,2.286210e+07,8.875876e+05,74021.0,Base,L2,6.640788,7.514569,2024Q1,2024 Q1
4,2024-01-01,7.328073e+06,3.308773e+05,27181.0,Arbitrum,L2,2.128596,2.801301,2024Q1,2024 Q1
6,2024-01-01,6.635193e+05,2.752500e+04,15267.0,Optimism,L2,0.192733,0.233034,2024Q1,2024 Q1
11,2023-10-01,3.372263e+08,1.018415e+07,219847.0,Ethereum,L1,98.041533,96.560109,2023Q4,2023 Q4
9,2023-10-01,5.530552e+06,2.971159e+05,53935.0,Arbitrum,L2,1.607893,2.817077,2023Q4,2023 Q4


In [266]:
quarterly_data_sorted_vol

,QUARTER,VOLUME_USD,TOTAL_FEES_USD,UNIQUE_TOTAL_USERS,NETWORK,LAYER,PERCENTAGE_OF_TOTAL_VOLUME,PERCENTAGE_OF_TOTAL_FEES,QUARTER_PERIOD,QUARTER_STR
3,2024-04-01,1.464951e+08,4.000557e+06,119206.0,Ethereum,L1,60.947137,67.749780,2024Q2,2024 Q2
1,2024-04-01,8.832805e+07,1.746792e+06,224213.0,Base,L2,36.747597,29.582075,2024Q2,2024 Q2
2,2024-04-01,4.471819e+06,1.524340e+05,22008.0,Arbitrum,L2,1.860435,2.581483,2024Q2,2024 Q2
0,2024-04-01,1.069214e+06,5.117316e+03,8621.0,Optimism,L2,0.444831,0.086662,2024Q2,2024 Q2
5,2024-01-01,3.134141e+08,1.056557e+07,207228.0,Ethereum,L1,91.037882,89.451095,2024Q1,2024 Q1
7,2024-01-01,2.286210e+07,8.875876e+05,74021.0,Base,L2,6.640788,7.514569,2024Q1,2024 Q1
4,2024-01-01,7.328073e+06,3.308773e+05,27181.0,Arbitrum,L2,2.128596,2.801301,2024Q1,2024 Q1
6,2024-01-01,6.635193e+05,2.752500e+04,15267.0,Optimism,L2,0.192733,0.233034,2024Q1,2024 Q1
11,2023-10-01,3.372263e+08,1.018415e+07,219847.0,Ethereum,L1,98.041533,96.560109,2023Q4,2023 Q4
9,2023-10-01,5.530552e+06,2.971159e+05,53935.0,Arbitrum,L2,1.607893,2.817077,2023Q4,2023 Q4


In [267]:
total_unique_users = quarterly_data.groupby('QUARTER')['UNIQUE_TOTAL_USERS'].sum().reset_index()
total_unique_users.columns = ['QUARTER', 'TOTAL_UNIQUE_USERS']

# Merge total unique users back into the original DataFrame
quarterly_data = quarterly_data.merge(total_unique_users, on='QUARTER')

# Calculate the percentage of unique users for each network
quarterly_data['PERCENTAGE_OF_TOTAL_USERS'] = (quarterly_data['UNIQUE_TOTAL_USERS'] / quarterly_data['TOTAL_UNIQUE_USERS']) * 100



In [268]:
total_unique_users

,QUARTER,TOTAL_UNIQUE_USERS
0,2023-07-01,357685.0
1,2023-10-01,311021.0
2,2024-01-01,323697.0
3,2024-04-01,374048.0


In [269]:
quarterly_data

,QUARTER,VOLUME_USD,TOTAL_FEES_USD,UNIQUE_TOTAL_USERS,NETWORK,LAYER,PERCENTAGE_OF_TOTAL_VOLUME,PERCENTAGE_OF_TOTAL_FEES,QUARTER_PERIOD,QUARTER_STR,VOLUME_USD_CLEAN,TOTAL_UNIQUE_USERS,PERCENTAGE_OF_TOTAL_USERS
0,2024-04-01,1.069214e+06,5.117316e+03,8621.0,Optimism,L2,0.444831,0.086662,2024Q2,2024 Q2,1.1M,374048.0,2.304784
1,2024-04-01,8.832805e+07,1.746792e+06,224213.0,Base,L2,36.747597,29.582075,2024Q2,2024 Q2,88.3M,374048.0,59.942307
2,2024-04-01,4.471819e+06,1.524340e+05,22008.0,Arbitrum,L2,1.860435,2.581483,2024Q2,2024 Q2,4.5M,374048.0,5.883737
3,2024-04-01,1.464951e+08,4.000557e+06,119206.0,Ethereum,L1,60.947137,67.749780,2024Q2,2024 Q2,146.5M,374048.0,31.869172
4,2024-01-01,7.328073e+06,3.308773e+05,27181.0,Arbitrum,L2,2.128596,2.801301,2024Q1,2024 Q1,7.3M,323697.0,8.397050
5,2024-01-01,3.134141e+08,1.056557e+07,207228.0,Ethereum,L1,91.037882,89.451095,2024Q1,2024 Q1,313.4M,323697.0,64.019129
6,2024-01-01,6.635193e+05,2.752500e+04,15267.0,Optimism,L2,0.192733,0.233034,2024Q1,2024 Q1,663.5K,323697.0,4.716448
7,2024-01-01,2.286210e+07,8.875876e+05,74021.0,Base,L2,6.640788,7.514569,2024Q1,2024 Q1,22.9M,323697.0,22.867373
8,2023-10-01,9.692585e+05,5.110091e+04,23088.0,Base,L2,0.281792,0.484509,2023Q4,2023 Q4,969.3K,311021.0,7.423293
9,2023-10-01,5.530552e+06,2.971159e+05,53935.0,Arbitrum,L2,1.607893,2.817077,2023Q4,2023 Q4,5.5M,311021.0,17.341273


In [270]:
base_data = quarterly_data[quarterly_data['NETWORK'] == 'base']
eth_data = quarterly_data[quarterly_data['NETWORK'] == 'ethereum']

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces for percentage of unique users by network
for network in quarterly_data['NETWORK'].unique():
    network_df = quarterly_data[quarterly_data['NETWORK'] == network]
    fig.add_trace(
        go.Scatter(
            x=network_df['QUARTER'],
            y=network_df['UNIQUE_TOTAL_USERS'],
            name=f'{network}',
            mode='lines+markers',  # Ensure text is included
            text=network_df['UNIQUE_TOTAL_USERS'].apply(clean_values),
            textposition='middle center',  # Position of the text
            texttemplate='%{text}',  # Format of the text
            # stackgroup='one',
            # groupnorm='percent'
        ),
        secondary_y=False
    ) 

# Add annotations with manually adjusted y-values
for _, row in base_data.iterrows():
    fig.add_annotation(
        x=row['QUARTER'],
        y=row['UNIQUE_TOTAL_USERS'],  # Use normalized value for annotations
        text=clean_values(row['UNIQUE_TOTAL_USERS']),
        showarrow=True,
        arrowhead=2,
        ax=-20,
        ay=-30,
        font=dict(size=10),
        xref='x',
        yref='y'
    )

for _, row in eth_data.iterrows():
    fig.add_annotation(
        x=row['QUARTER'],
        y=row['UNIQUE_TOTAL_USERS'],  # Use normalized value for annotations
        text=clean_values(row['UNIQUE_TOTAL_USERS']),
        showarrow=True,
        arrowhead=2,
        ax=-20,
        ay=-30,
        font=dict(size=10),
        xref='x',
        yref='y'
    )
    

fig.update_layout(
    title='Number of Unique Users By Network',
    xaxis_title='Quarter',
    yaxis_title='Number of Users',
    legend=dict(yanchor="top", y=0.5, xanchor="left", x=1.05),
    plot_bgcolor='#f4f4f4',
    paper_bgcolor='#f4f4f4'
)

fig.show()

In [271]:
quarterly_data['QUARTER_STR']

0     2024 Q2
1     2024 Q2
2     2024 Q2
3     2024 Q2
4     2024 Q1
5     2024 Q1
6     2024 Q1
7     2024 Q1
8     2023 Q4
9     2023 Q4
10    2023 Q4
11    2023 Q4
12    2023 Q3
13    2023 Q3
14    2023 Q3
15    2023 Q3
Name: QUARTER_STR, dtype: object

In [272]:
quarterly_data = quarterly_data.sort_values('QUARTER', ascending=True)

In [273]:
# Create the subplot figure
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces for percentage of unique users by network
for network in legend_order:
    network_df = quarterly_data[quarterly_data['NETWORK'] == network]
    fig.add_trace(
        go.Scatter(
            x=network_df['QUARTER_STR'],
            y=network_df['UNIQUE_TOTAL_USERS'],
            name=f'{network}',
            mode='lines+markers',
            text=network_df['UNIQUE_TOTAL_USERS'].apply(clean_values),
            textposition='middle center',
            texttemplate='%{text}',
            line=dict(
                width=3  # Line width (thicker line)
            ),
            marker=dict(
                color=color_map.get(network, 'black'),
                size=10  # Marker size (larger markers)
            )
        ),
        secondary_y=False
    )

# Add annotations for the beginning and end of each trace
annotations = []
for network in ['Base', 'Ethereum']:
    network_df = quarterly_data[quarterly_data['NETWORK'] == network]
    first_row = network_df.iloc[-1]
    last_row = network_df.iloc[0]
    
    # Add annotations for the start and end points
    annotations.append(dict(
        x=first_row['QUARTER_STR'],
        y=first_row['UNIQUE_TOTAL_USERS'],
        text=f'{network} Q2 24: {clean_values(first_row["UNIQUE_TOTAL_USERS"])}',
        showarrow=True,
        arrowhead=2,
        ax=-50,
        ay=-30,
        font=dict(size=25, family='Arial, sans-serif', weight='bold', color='black'),
        xref='x',
        yref='y'
    ))
    annotations.append(dict(
        x=last_row['QUARTER_STR'],
        y=last_row['UNIQUE_TOTAL_USERS'],
        text=f'{network} Q3 23: {clean_values(last_row["UNIQUE_TOTAL_USERS"])}',
        showarrow=True,
        arrowhead=2,
        ax=100,
        ay=-50,
        font=dict(size=25, family='Arial, sans-serif', weight='bold', color='black'),
        xref='x',
        yref='y'
    ))


# Update layout with annotations
fig.update_layout(
    xaxis_title='Quarter',
    yaxis_title='Number of Users',
    legend=dict(yanchor="top", y=0.5, xanchor="left", x=1.05),
    plot_bgcolor='#f4f4f4',
    paper_bgcolor='#f4f4f4',
    annotations=annotations,
    uniformtext=dict(mode="show", minsize=25),  # Adjust minsize as needed
    font=dict(size=25, family="Arial, sans-serif")
)

# fig.update_layout(
#     title={
#         'text': 'Number of Unique OpenSea Users By Network',
#         'font': {
#             'family': 'Arial, sans-serif',  # You can specify the font family
#             'size': 24,  # Adjust the font size as needed
#             'color': 'Black'  # Adjust the font color as needed
#         }
#     }
# )

fig.update_xaxes(
    tickvals=quarterly_data['QUARTER_STR'],
    ticktext=quarterly_data['QUARTER_STR'],
    tickangle=-45,  # Optional: Rotate labels if needed
    # range=[quarterly_data['QUARTER_STR'].min(), quarterly_data['QUARTER_STR'].max()]  # Set x-axis range
)

fig.update_layout(
    width=1500,  # Adjust as needed
    height=800,  # Adjust as needed
    margin=dict(
        l=50,  # Left margin
        r=50,  # Right margin
        t=100,  # Top margin
        b=50   # Bottom margin
    )
)

fig.update_layout(
    xaxis_title=dict(
        text="Quarter",
        font=dict(size=25, family="Arial, sans-serif", weight="bold", color='black')
    ),
    yaxis_title=dict(
        text="Number of Users",
        font=dict(size=25, family="Arial, sans-serif", weight="bold", color='black')
    )
)

fig.show()

pio.write_image(fig, 'img/Number of Unique OpenSea Users By Network.png', scale=3)


In [274]:
quarterly_data.sort_values(by=['NETWORK', 'QUARTER'], inplace=True)
quarterly_data['PCT_CHANGE_UNIQUE_USERS'] = quarterly_data.groupby('NETWORK')['UNIQUE_TOTAL_USERS'].pct_change() * 100
quarterly_data['PCT_CHANGE_UNIQUE_USERS'].fillna(0,inplace=True)
print(quarterly_data[['QUARTER','NETWORK','UNIQUE_TOTAL_USERS','PCT_CHANGE_UNIQUE_USERS']])


      QUARTER   NETWORK  UNIQUE_TOTAL_USERS  PCT_CHANGE_UNIQUE_USERS
14 2023-07-01  Arbitrum             22758.0                 0.000000
9  2023-10-01  Arbitrum             53935.0               136.993585
4  2024-01-01  Arbitrum             27181.0               -49.604153
2  2024-04-01  Arbitrum             22008.0               -19.031677
12 2023-07-01      Base             42927.0                 0.000000
8  2023-10-01      Base             23088.0               -46.215668
7  2024-01-01      Base             74021.0               220.603777
1  2024-04-01      Base            224213.0               202.904581
13 2023-07-01  Ethereum            245918.0                 0.000000
11 2023-10-01  Ethereum            219847.0               -10.601501
5  2024-01-01  Ethereum            207228.0                -5.739901
3  2024-04-01  Ethereum            119206.0               -42.475920
15 2023-07-01  Optimism             46082.0                 0.000000
10 2023-10-01  Optimism           

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_34072\1135001275.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [275]:
quarterly_totals_layers = quarterly_data.groupby(['QUARTER', 'LAYER'])[['VOLUME_USD', 'TOTAL_FEES_USD', 'UNIQUE_TOTAL_USERS']].sum()
quarterly_totals_layers = quarterly_totals_layers.reset_index()
quarterly_totals_layers

,QUARTER,LAYER,VOLUME_USD,TOTAL_FEES_USD,UNIQUE_TOTAL_USERS
0,2023-07-01,L1,2.108915e+08,9.204380e+06,245918.0
1,2023-07-01,L2,2.993722e+06,2.444479e+05,111767.0
2,2023-10-01,L1,3.372263e+08,1.018415e+07,219847.0
3,2023-10-01,L2,6.736397e+06,3.628039e+05,91174.0
4,2024-01-01,L1,3.134141e+08,1.056557e+07,207228.0
5,2024-01-01,L2,3.085369e+07,1.245990e+06,116469.0
6,2024-04-01,L1,1.464951e+08,4.000557e+06,119206.0
7,2024-04-01,L2,9.386908e+07,1.904343e+06,254842.0


In [276]:
quarterly_totals_layers['QUARTER'] = pd.to_datetime(quarterly_totals_layers['QUARTER'])

# Sort by LAYER and QUARTER
quarterly_totals_layers_sorted = quarterly_totals_layers.sort_values(by=['LAYER', 'QUARTER'])

# Calculate the percentage change in VOLUME_USD within each LAYER group
quarterly_totals_layers_sorted['PCT_CHANGE_VOL'] = (
    quarterly_totals_layers_sorted.groupby('LAYER')['VOLUME_USD']
    .pct_change() * 100
)

# Display relevant columns
quarterly_totals_layers_sorted['PCT_CHANGE_VOL'].fillna(0, inplace=True)
print(quarterly_totals_layers_sorted[['QUARTER', 'LAYER', 'VOLUME_USD', 'PCT_CHANGE_VOL']])

     QUARTER LAYER    VOLUME_USD  PCT_CHANGE_VOL
0 2023-07-01    L1  2.108915e+08        0.000000
2 2023-10-01    L1  3.372263e+08       59.905118
4 2024-01-01    L1  3.134141e+08       -7.061171
6 2024-04-01    L1  1.464951e+08      -53.258310
1 2023-07-01    L2  2.993722e+06        0.000000
3 2023-10-01    L2  6.736397e+06      125.017460
5 2024-01-01    L2  3.085369e+07      358.014699
7 2024-04-01    L2  9.386908e+07      204.239427


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_34072\3644702449.py:13: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [277]:
total_volume_per_quarter = quarterly_totals_layers.groupby('QUARTER')['VOLUME_USD'].sum().reset_index()
total_volume_per_quarter.rename(columns={'VOLUME_USD': 'TOTAL_VOLUME_USD'}, inplace=True)

# Merge this total back with the original DataFrame
merged_df = pd.merge(quarterly_totals_layers, total_volume_per_quarter, on='QUARTER')

# Step 2: Calculate percentage of total volume by layer by quarter
merged_df['PERCENTAGE_OF_TOTAL_VOLUME'] = (merged_df['VOLUME_USD'] / merged_df['TOTAL_VOLUME_USD']) * 100

# Select relevant columns
result = merged_df[['QUARTER', 'LAYER', 'PERCENTAGE_OF_TOTAL_VOLUME']]
print(result)

     QUARTER LAYER  PERCENTAGE_OF_TOTAL_VOLUME
0 2023-07-01    L1                   98.600314
1 2023-07-01    L2                    1.399686
2 2023-10-01    L1                   98.041533
3 2023-10-01    L2                    1.958467
4 2024-01-01    L1                   91.037882
5 2024-01-01    L2                    8.962118
6 2024-04-01    L1                   60.947137
7 2024-04-01    L2                   39.052863


bubble chart of top 5 projects by volume by network in past month; color is layer, bubble size ?, x = amt,

In [278]:
merged_df

,QUARTER,LAYER,VOLUME_USD,TOTAL_FEES_USD,UNIQUE_TOTAL_USERS,TOTAL_VOLUME_USD,PERCENTAGE_OF_TOTAL_VOLUME
0,2023-07-01,L1,2.108915e+08,9.204380e+06,245918.0,2.138852e+08,98.600314
1,2023-07-01,L2,2.993722e+06,2.444479e+05,111767.0,2.138852e+08,1.399686
2,2023-10-01,L1,3.372263e+08,1.018415e+07,219847.0,3.439627e+08,98.041533
3,2023-10-01,L2,6.736397e+06,3.628039e+05,91174.0,3.439627e+08,1.958467
4,2024-01-01,L1,3.134141e+08,1.056557e+07,207228.0,3.442678e+08,91.037882
5,2024-01-01,L2,3.085369e+07,1.245990e+06,116469.0,3.442678e+08,8.962118
6,2024-04-01,L1,1.464951e+08,4.000557e+06,119206.0,2.403642e+08,60.947137
7,2024-04-01,L2,9.386908e+07,1.904343e+06,254842.0,2.403642e+08,39.052863


In [279]:
projects_by_vol.columns

Index(['PROJECT_NAME', 'NFT_ADDRESS', 'QUARTER', 'VOLUME', 'AVERAGE_SALE',
       'TOTAL_FEES', 'NETWORK', 'LAYER'],
      dtype='object')

In [280]:
projects_by_vol['PROJECT_NAME'].unique()

array(['higher swatches', 'BOSS', 'based punks', 'BoredApeYachtClub',
       'MutantApeYachtClub', 'onchain gaias', 'swatches by jvmi',
       'Art Blocks', 'PudgyPenguins', 'OnChainShiba',
       'Captain & Company - Ships', 'lueygi_KEY', 'OP Dragons',
       'BTC Girls X', 'Star Heroes', 'ZTX Genesis Homes', 'Yeetard NFTs',
       '3DNS Powered Domain Name', 'Skeleton ERA 💀', 'Cat'], dtype=object)

fig = px.scatter(projects_by_vol, x="VOLUME", y="AVERAGE_SALE",
	         size="VOLUME", color="NETWORK",
                 hover_name="PROJECT_NAME", log_x=True, size_max=60, text='PROJECT_NAME')

for i, row in projects_by_vol.iterrows():
    fig.add_annotation(
        x=row['VOLUME'],
        y=row['AVERAGE_SALE'],
        text=row['PROJECT_NAME'],  # Add project name and layer
        showarrow=True,  # Ensure the arrow is shown
        arrowhead=2,  # Style of the arrowhead
        ax=0,  # Arrow x offset (adjust as needed)
        ay=-30,  # Arrow y offset (adjust as needed)
        font=dict(size=10),  # Font size
        align='center'  # Align text
    )

fig.update_layout(
    title="Top NFT Collections by Volume, Q2 2024",
    xaxis_title="Volume (USD)",
    yaxis_title="Average Sale Price (USD)",
    showlegend=True
)


In [281]:
def clean_value(value):
    if value >= 1e9:
        return f"{value / 1e9:.1f}b"
    elif value >= 1e6:
        return f"{value / 1e6:.1f}m"
    elif value >= 1e3:
        return f"{value / 1e3:.1f}k"
    else:
        return f"{value:.0f}"

In [282]:
network_mapping = {network: idx for idx, network in enumerate(projects_by_vol['NETWORK'].unique())}
projects_by_vol['NETWORK_NUM'] = projects_by_vol['NETWORK'].map(network_mapping)

max_volume = projects_by_vol['VOLUME'].max()
sizeref = 0.15 * max_volume / (40.**2)  # Scale marker sizes

# Function to check overlap
def is_overlapping(annotation, used_annotations, threshold=50):
    for ann in used_annotations:
        if abs(annotation['x'] - ann['x']) < threshold and abs(annotation['y'] - ann['y']) < threshold:
            return True
    return False


fig = go.Figure()

# Add traces for each network
for network in legend_order:
    network_df = projects_by_vol[projects_by_vol['NETWORK'] == network]
    fig.add_trace(
        go.Scatter(
            x=network_df['VOLUME'],
            y=network_df['AVERAGE_SALE'],  # Use average sale price as y-axis
            mode='markers+text',
            name=network,
            marker=dict(
                size=network_df['VOLUME'],  # Set size based on VOLUME
                sizemode='area',
                sizeref=sizeref,  # Reference for scaling
                sizemin=6,
                color=color_map.get(network, 'black')
            ),
            
            # text=network_df['PROJECT_NAME'],
            # textposition='top center',
            # textfont=dict(size=8)
            
        )
    )

used_annotations = []
# Manually set annotations for largest and smallest data points, and for a specific collection
for network in projects_by_vol['NETWORK'].unique():
    network_df = projects_by_vol[projects_by_vol['NETWORK'] == network]

    # Find the largest and smallest points
    largest = network_df.loc[network_df['VOLUME'].idxmax()]
    smallest = network_df.loc[network_df['VOLUME'].idxmin()]

    network_df = network_df[network_df['PROJECT_NAME'] != 'OnChainShiba']
    
    
    
    # Define manual adjustments for each point
    adjustments = {
        'largest': {'ax': 20, 'ay': -40},
        'smallest': {'ax': -20, 'ay': 30}
    }
    
    for point, adjustment in zip([largest, smallest], ['largest', 'smallest']):
        if point['PROJECT_NAME'] in(['OnChainShiba','OP Dragons']):
            continue  # Skip OnChainShiba
        x_pos = math.log10(point['VOLUME'])
        y_pos = point['AVERAGE_SALE']
        
        annotation = dict(
            x=x_pos,
            y=y_pos,
            text=f"{point['PROJECT_NAME']} ({clean_value(point['VOLUME'])})",
            showarrow=True,
            arrowhead=2,
            ax=adjustments[adjustment]['ax'],
            ay=adjustments[adjustment]['ay'],
            font=dict(size=20, color='black', family='Arial, sans-serif', weight='bold'),
            bgcolor='rgba(255, 255, 255, 0.8)',
            bordercolor='black',
            borderwidth=1
        )
        
        used_annotations.append(annotation)
        fig.add_annotation(annotation)

# Add specific annotation for "onchainshiba"
onchainshiba_df = projects_by_vol[projects_by_vol['PROJECT_NAME'] == 'OnChainShiba']
if not onchainshiba_df.empty:
    point = onchainshiba_df.iloc[0]
    x_pos = math.log10(point['VOLUME'])
    y_pos = point['AVERAGE_SALE']
    
    fig.add_annotation(
        x=x_pos,
        y=y_pos,
        text=f"{point['PROJECT_NAME']} ({clean_value(point['VOLUME'])})",
        showarrow=True,
        arrowhead=2,
        ax=-30,  # Manually set ax for "onchainshiba"
        ay=-30, # Manually set ay for "onchainshiba"
        font=dict(size=20, color='black', family='Arial, sans-serif', weight='bold'),
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='black',
        borderwidth=1
    )

# Add specific annotation for "onchainshiba"
opdragons_df = projects_by_vol[projects_by_vol['PROJECT_NAME'] == 'OP Dragons']
if not opdragons_df.empty:
    point = opdragons_df.iloc[0]
    x_pos = math.log10(point['VOLUME'])
    y_pos = point['AVERAGE_SALE']
    
    fig.add_annotation(
        x=x_pos,
        y=y_pos,
        text=f"{point['PROJECT_NAME']} ({clean_value(point['VOLUME'])})",
        showarrow=True,
        arrowhead=2,
        ax=-40,  # Manually set ax for "onchainshiba"
        ay=-60, # Manually set ay for "onchainshiba"
        font=dict(size=20, color='black', family='Arial, sans-serif', weight='bold'),
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='black',
        borderwidth=1
    )


# Update layout
fig.update_layout(
    showlegend=True,
    xaxis_type="log",
    xaxis=dict(
        tickmode='array',  # Use this to set specific ticks
        tickvals=[1e4, 1e5, 1e6, 1e7, 5e7],  # Example tick values (adjust as needed)
        ticktext=['10k', '100k', '1m', '10m', '20m']  # Custom tick labels
    ),
    yaxis=dict(
        range=[-10000, projects_by_vol['AVERAGE_SALE'].max() * 1.15],  # Provide space below
        tickvals=[0, 10000, 20000,30000,40000,50000,60000],  # Customize tick values
        ticktext=['0', '10k', '20k', '30k', '40k', '50k', '60k']  # Customize tick labels to avoid negative values
    ),
    
    uniformtext=dict(mode="show", minsize=20),  # Adjust minsize as needed
    font=dict(size=20, family="Arial, sans-serif"),
    plot_bgcolor='#f4f4f4',
    paper_bgcolor='#f4f4f4'
)

fig.update_layout(
    xaxis_title=dict(
        text="Sales Volume (USD)",
        font=dict(size=25, family="Arial, sans-serif", weight="bold", color='black')
    ),
    yaxis_title=dict(
        text="Average Sale Price (USD)",
        font=dict(size=25, family="Arial, sans-serif", weight="bold", color='black')
    )
)
# fig.update_layout(
#     title={
#         'text': 'Top OpenSea Collections by Volume, Q2 2024',
#         'font': {
#             'family': 'Arial, sans-serif',  # You can specify the font family
#             'size': 24,  # Adjust the font size as needed
#             'color': 'Black'  # Adjust the font color as needed
#         }
#     }
# )

fig.update_layout(
    width=1200,  # Adjust as needed
    height=600,  # Adjust as needed
    # margin=dict(
    #     l=50,  # Left margin
    #     r=50,  # Right margin
    #     t=100,  # Top margin
    #     b=50   # Bottom margin
    # )
)

fig.show()

pio.write_image(fig, 'img/Top OpenSea Collections By Volume.png', scale=3)





Can analyze vm.box domain sale as largest sale for 3dns in last quarter 

"The highest sold on-chain DNS domain sale to date is watch.box, sold on May 6, 2024, for 23.59 ETH / $73,644, from the 3DNS collection.  Blockexplorer: https://optimistic.etherscan.io/tx/0xb18bea2b0f22cfbc496acae79853a3b974c3afc6fe99f564d648b3499bce4df7 

Get uniform colors for each layer, each network